# Capture faces via webcam

## Instructions to use:

1. Run the cell.
2. Input name in lower case without spacing. If spacing is required, use underscore instead.
3. Take these 5 face expressions:
        - No glasses, no smile.
        - No glasses, smile, no show teeth.
        - No glasses, smile, show teeth.
        - Wear glasses, no smile.
        - Wear glasses, smile, show teeth.
4. Position face in front of webcam so that "FACE" and "EYES" are detected. Then press "spacebar" to capture face image.
5. Repeat step 3 for other face expressions.
6. Press "ESC" to capture different person face.
7. Colored face images with roi will be saved in "ROI_Faces" folder.
8. Grayscale face images without roi, with size (100,100) will be saved in "Grayscale_Faces" folder for training the face recognizer.
9. Colored face images without roi, with size (224,224) will be saved in "Color_Faces" folder for training the face recognizer.

In [2]:
import os, sys
import cv2
from PIL import Image
import numpy as np
import pickle
import time
import matplotlib.pyplot as plt
%matplotlib inline

# Take face shots via webcam to create face database
cam = cv2.VideoCapture(0)
cam.set(3, 1280) # set video width
cam.set(4, 960) # set video height
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eyes_detector = cv2.CascadeClassifier('haarcascade_eye.xml')
eyes_glasses_detector = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
# For each person, enter one numeric face id
user_name = input('\n Enter name and press <enter> ==>  ')
print("\n Look at camera and press <spacebar> to take picture")
# Initialize individual sampling face count
count = 1
while(True):
    ret, img = cam.read()
    img = cv2.flip(img, 1) # flip video image vertically
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(5,5))
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        resized_color = cv2.resize(img[y:y+h,x:x+w], dsize=(224,224), interpolation=cv2.INTER_AREA)
        eyes = eyes_glasses_detector.detectMultiScale(roi_gray, scaleFactor=1.2, minNeighbors=5, minSize=(5,5));
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
            #cv2.rectangle(roi_gray, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
        resized_gray = cv2.resize(gray[y:y+h,x:x+w], dsize=(100,100), interpolation=cv2.INTER_AREA)
        
    k = cv2.waitKey(1) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif k == 32: #if spacebar is pressed
        # Save the captured image into the datasets folder
        cv2.imwrite("ROI_Faces/User." + str(user_name).lower() + "." + str(count) + ".jpg", roi_color)
        cv2.imwrite("Grayscale_Faces/User." + str(user_name).lower() + '.'+ str(count) + ".jpg", resized_gray)
        cv2.imwrite("Color_Faces/User." + str(user_name).lower() + '.' + str(count) + ".jpg", resized_color)
        print("Shot {} is taken!".format(count))
        count += 1  
    cv2.imshow('image', img)

cam.release()
cv2.destroyAllWindows()

KeyboardInterrupt: Interrupted by user

# Load Face Images 

In [1]:
def get_faces_labels(resized_images_path='Resized_Faces'):
    file_path = os.listdir(resized_images_path)
    faces = []
    face_labels = []
    current_id=0
    label_ids={}
    for file in file_path:
        if file.endswith("png") or file.endswith("jpg"):
            path=os.path.join(resized_images_path, file)
            label=os.path.basename(path).split(".")[1]
            #print(label,path)
            if not label in label_ids:
                #label_ids[label] = os.path.basename(path).split(".")[2]
                label_ids[label]=current_id
                current_id+=1

            id_=label_ids[label]
            pil_image=Image.open(path)#grayscale
            image_array=np.array(pil_image,'uint8')
                           
            #print(image_array)
            faces.append(image_array)
            face_labels.append(id_)
            faces_array = np.array(faces)
            labels_array = np.array(face_labels)
    return faces_array, labels_array, label_ids

In [3]:
# Load labelled face datasets
face, labels, name_label_dict = get_faces_labels()
print("Shape of Training Faces: ",face.shape)
print("########################################")
print("Shape of Face Labels: ",labels.shape)
print("########################################")
print("Dictionary of Person Name and Label: ",name_label_dict)
print("########################################")
print("Labels: ",labels)

Shape of Training Faces:  (100, 100, 100)
########################################
Shape of Face Labels:  (100,)
########################################
Dictionary of Person Name and Label:  {'bryan_lee': 0, 'bryan_lim': 1, 'chin_fung': 2, 'darren': 3, 'edmund': 4, 'john': 5, 'lam': 6, 'malvern': 7, 'meinv': 8, 'nicholas': 9, 'peter': 10, 'terren': 11, 'wangjue': 12, 'yicheng': 13, 'yirong': 14, 'yongzhe': 15, 'yuanjun': 16, 'zhijia': 17, 'zihang': 18, 'ziying': 19}
########################################
Labels:  [ 0  0  0  0  0  1  1  1  1  1  2  2  2  2  2  3  3  3  3  3  4  4  4  4
  4  5  5  5  5  5  6  6  6  6  6  7  7  7  7  7  8  8  8  8  8  9  9  9
  9  9 10 10 10 10 10 11 11 11 11 11 12 12 12 12 12 13 13 13 13 13 14 14
 14 14 14 15 15 15 15 15 16 16 16 16 16 17 17 17 17 17 18 18 18 18 18 19
 19 19 19 19]


# Train LBPH Face Recognizer

In [ ]:
# Train face recognizer and save trained recognizer.
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(face, labels)
face_recognizer.save("LBPH_Face_Recognizer.yml")

# Real-Time Face Recognition

In [5]:
def lbph_face_recognition():
    faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') # load classifier
    eyeCascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    eye_glassesCascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
    smileCascade = cv2.CascadeClassifier('haarcascade_smile.xml')
    #faceCascade = cv2.CascadeClassifier('lbpcascade_frontalface.xml') # load classifier

    cap = cv2.VideoCapture(0)
    #face_recognizer = cv2.face.EigenFaceRecognizer_create()
    face_recognizer=cv2.face.LBPHFaceRecognizer_create()
    face_recognizer.read("LBPH_Face_Recognizer.yml")

    #Load face labels
    # with open('face_eye_smile_labels.pickle', 'rb') as f:
    #     y_train = np.array(pickle.load(f))
    names = ['Bryan_Lee', 'Bryan_Lim', 'Chin_Fung', 'Darren', 'Edmund', 'John', 'Lam', 
             'Malvern', 'meinv', 'Nicholas', 'Peter', 'Ter_Ren', 'Wang_Jue', 'Yi_Cheng', 
             'Yi_Rong', 'Yong_Zhe', 'Yuan_Jun', 'Zhi_jia', 'Zi_Hang', 'Zi_Ying']
    
    cap.set(3,640) # set Width
    cap.set(4,480) # set Height
    while True:
        _, frame = cap.read()
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # detect faces
        faces = faceCascade.detectMultiScale(
            gray,     
            scaleFactor=1.2,
            minNeighbors=5,     
            minSize=(5, 5)
        )
        found=0
        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            roi_color = frame[y:y+h, x:x+w]
            roi_gray = gray[y:y+h, x:x+w]
    #        roi_gray = cv2.resize(roi_gray, (100,100))
            eyes = eye_glassesCascade.detectMultiScale(roi_gray, scaleFactor=1.2, minNeighbors=5, minSize=(5,5))
            for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
    #         smile = smileCascade.detectMultiScale(roi_gray, scaleFactor=1.2, minNeighbors=25, minSize=(120,120))
    #         for (xx, yy, ww, hh) in smile:
    #             cv2.rectangle(roi_color, (xx, yy), (xx + ww, yy + hh), (0, 0, 255), 2)
            gray_face = cv2.resize((gray[y:y+h,x:x+w]),(100,100))
            label, conf = face_recognizer.predict(gray_face)
            
            found=1
            
        if (found==1):

            if conf<=120:
                person = names[label]
            else:
                person = "Unknown"

            text = str(label) + person + ":" + str(round(conf,3))
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        elif (found==0):
            eyes = eye_glassesCascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(5,5))
            for (ex, ey, ew, eh) in eyes:
                roi_gray = gray[ey:ey+eh, ex:ex+ew]
                cv2.rectangle(frame, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
        #         smile = smileCascade.detectMultiScale(roi_gray, scaleFactor=1.2, minNeighbors=25, minSize=(120,120))
        #         for (xx, yy, ww, hh) in smile:
        #             cv2.rectangle(roi_color, (xx, yy), (xx + ww, yy + hh), (0, 0, 255), 2)
            #gray_eyes = cv2.resize((gray[y:y+h,x:x+w]),(100,100))
                label, conf = face_recognizer.predict(gray)
            if conf<=120:
                person = names[label]

            else:
                person = "Unknown"

            text = str(label) + person + ":" + str(round(conf,3))
            cv2.rectangle(frame, (ex, ey), (ex + ew, ey + eh), (255, 0, 0), 2)
            cv2.putText(frame, text, (ex, ey - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        cv2.imshow('frame',frame)
        k = cv2.waitKey(1) & 0xff
        if k == 27: # press 'ESC' to quit
            break
    cap.release()
    cv2.destroyAllWindows()

In [6]:
lbph_face_recognition()